# Problem set 3: Loading and structuring data from Denmark Statistics

In [1]:
# Import modeles 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display


# Set styles, formats and magic
pd.options.display.float_format = '{:,.3f}'.format
%matplotlib inline
plt.style.use("seaborn-v0_8-whitegrid")

# Tasks

## Create a pandas DataFrame

**Question:**

Modify the code below such that *income* and *consumption* are variables in the *dt* DataFrame.

In [ ]:
np.random.seed(1999)

N = 100
mydata = {}
mydata['id'] = range(N)
income = np.exp(np.random.normal(size=N))
consumption = np.sqrt(income)

dt = pd.DataFrame(mydata)
dt.head()

**Answer:**
<br>
See below or A01


The slow way:

In [ ]:
# Set a seed for specifying income and consumption
np.random.seed(1999)

# Set number of values in range
N = 100

# Create a dictionary
mydata = {}

# Create a key for the id and assign id-values of range(N)
mydata['id'] = range(N)

# Create psuedo-random values for income and consumption
income = np.exp(np.random.normal(size=N))
consumption = np.sqrt(income)

# Create keys for income and consumption and assign values
mydata['income'] = income
mydata['consumption'] = consumption

# Use the dictionary 'mydata' to create a pd.DataFrame
dt = pd.DataFrame(mydata)

# Display the head(implicitly the first 5 rows) of the DataFrame
dt.head()

The fast way:

In [ ]:
# Set a seed for specifying income and consumption
np.random.seed(1999)

# Set number of values in range
N = 100

# Create a dictionary
mydata = {}

# Create a key for the id and assign id-values of range(N)
mydata['id'] = range(N)

# Create a key for the income and assign values from a standard normal distribution
mydata['income'] = np.exp(np.random.normal(size=N))

# Create a key for consumption with values as a function of the income key values
mydata['consumption'] = np.sqrt(mydata['income'])

# Use the dictionary 'mydata' to create a pd.DataFrame
dt = pd.DataFrame(mydata)

# Display the head(implicitly the first 5 rows) of the DataFrame
dt.head()

## Create new variable

**Question:**
<br>
Add a new variable *ratio* which is the ratio of consumption to income

**Answer:**
<br>
See below or A02

The slow way:

In [ ]:
# Create a key for ratio with values as a function of the consumption and income key values
mydata['ratio'] = mydata['consumption'] / mydata['income']

# Use the dictionary 'mydata' to create a pd.DataFrame
dt = pd.DataFrame(mydata)

# Display the head(implicitly the first 5 rows) of the DataFrame
dt.head()

The fast way:

In [ ]:
# Create the pd.DataFrame column ratio as function of the columns consumption and income
dt['ratio'] = dt['consumption'] / dt['income']

# Display the head(implicitly the first 5 rows) of the DataFrame
dt.head()

## Summary statistics

**Question:**
<br>
Produce summary statistics using `.describe()`.

**Answer:**
<br>
See below or A03


Display summary statistics with a lot of decimals

In [ ]:
# Display summary statistics
dt.describe()

## Indexing

**Question:**
<br>
Select everybody with an income above 1.

**Answer:**
<br>
See below or A04


In [ ]:
# Create a (boolean) series of True/False values given the condition
I = dt['income'] > 1

# Display head of the pandas DataFrame where the condition holds
dt.loc[I,:].head()

**Question:**
<br>
Select everybody with an income *above* 1 and a ratio *above* 0.7.

**Answer:**
<br>
See below or A05


In [ ]:
# Create a (boolean) series of True/False values given the condition
IR = (dt['income'] > 1) & (dt['ratio'] > 0.7)

# Display head of the pandas DataFrame where the condition holds
dt.loc[IR,:].head()

**Question:**
<br>
Set consumption equal to 0.5 if income is less than 0.5.

**Answer:**
<br>
See below or A06

In [ ]:
# Create a (boolean) series of True/False values given the condition
I = dt['income'] < 0.5

# For all rows for which the condition holds true set consumption to 0.5
dt.loc[I, ['consumption']] = 0.5

# Find new mean of consumption
dt['consumption'].mean() # <- compare with answer

The mean consumption is less than before the change in consumption due to the condition

**Question:**
<br>
Set consumption equal to income if income is less than 0.5.

**Answer:**
<br>
See below or A07

In [ ]:
# Create a (boolean) series of True/False values given the condition
I = dt['income'] < 0.5

# For all rows for which the condition holds true set consumption to income values
dt.loc[I, ['consumption']] = dt.loc[I, ['income']].values

# Find new mean of consumption
dt['consumption'].mean() # <- compare with answer

The mean consumption is less than before the change in consumption due to the condition

## Dropping

**Question:**
<br>
Drop the *ratio* variable and all rows with an income above 1.5. After this, also drop the first 5 rows.

**Answer:**
<br>
See below or A08

To solve this I use the parameter `inplace = True` inside of the function `drop`, this parameter modifies the data directly or 'in place', eliminating the need to assign the modified data to a new variable

In [ ]:
print(f'before: {dt.shape[0]} observations, {dt.shape[1]} variables')

# Drop the column ratio
dt.drop(columns = ['ratio'], inplace = True)

# Create a (boolean) series of True/False values given the condition
I = dt['income'] > 1.5

# Drop rows for which the condition is true
dt.drop(dt.loc[I, :].index, inplace = True)

# Drop the first 5 rows
dt.drop(dt.loc[0:5, :].index, inplace = True)

print(f'after: {dt.shape[0]} observations, {dt.shape[1]} variables')

## Renaming

**Question:**
<br>
Rename *consumption* to *cons* and *income* to *inc*.

**Answer:**
<br>
See below or A09

In [ ]:
# Rename the columns
dt.rename(columns = {'consumption': 'cons', 'income': 'inc'}, inplace = True)

# Display the head of the DataFrame
dt.head()

## Functions

**Question:**
<br>
Correct the wrong lines such that `assets_1 = assets_2 = assets_3 = assets_4`.

**Answer:**
<br>
See below or A10

In [ ]:
# Define variables for the return rate and income
R = 1.2
Y = 1

# Calculate asset row by row
def assets_row_by_row(x,R,Y):
    return R * (x['inc'] - x['cons']) + Y

# Calculate asset for all rows at once 
def assets_all_at_once(income,consumption,R,Y):
    return R * (income - consumption) + Y

# Adjust asset in place 
def assets_adj(assets,R,Y):
    assets *= R
    assets += Y

# Test if asset_1 = ... = asset_4
try:
    dt['assets_1'] = R*(dt['inc']-dt['cons'])+Y
    dt['assets_2'] = dt.apply(assets_row_by_row, axis=1, args=(R,Y))
    dt['assets_3'] = assets_all_at_once(dt['inc'].values,dt['cons'].values,R,Y)
    dt['assets_4'] = dt['inc'] - dt['cons']
    assets_adj(dt['assets_4'], R, Y)
except:
    print('failed')

# Display the head of the DataFrame
dt.head()

**Answer:** see A10.py

# Problem

**Question:**
<br>
Load the data set in *data/NAH1_pivoted.xlsx* and clean and structure it such that the `plot_timeseries(dataframe)` below can be run and produce an interactive figure. 

In [ ]:
def _plot_timeseries(dataframe, variable, years):
    
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    
    dataframe.loc[:,['year']] = pd.to_numeric(dataframe['year'])
    I = (dataframe['year'] >= years[0]) & (dataframe['year'] <= years[1])
    
    x = dataframe.loc[I,'year']
    y = dataframe.loc[I,variable]
    ax.plot(x,y)
    
    ax.set_xticks(list(range(years[0], years[1] + 1, 5)))    
    
def plot_timeseries(dataframe):
    
    widgets.interact(_plot_timeseries, 
    dataframe = widgets.fixed(dataframe),
    variable = widgets.Dropdown(
        description='variable', 
        options=['Y','C','G','I','X','M'], 
        value='Y'),
    years=widgets.IntRangeSlider(
        description="years",
        min=1966,
        max=2018,
        value=[1980, 2018],
        continuous_update=False,
    )                 
); 

In [ ]:
rename_dict = {}
rename_dict['P.1 Output'] = 'Y'
rename_dict['P.3 Final consumption expenditure'] = 'C'
rename_dict['P.3 Government consumption expenditure'] = 'G'
rename_dict['P.5 Gross capital formation'] = 'I'
rename_dict['P.6 Export of goods and services'] = 'X'
rename_dict['P.7 Import of goods and services'] = 'M'

**Answer:**
<br>
See below or A10

In [ ]:
# Create path as a variable
path_nah1 = 'NAH1_pivoted.xlsx'

# Load data set and skip to two first rows
nah1 = pd.read_excel(path_nah1, skiprows = 2)

# Add 'Unnamed: 0' and 'Unnamed: 1' to rename_dict
rename_dict['Unnamed: 0'] = 'year'
rename_dict['Unnamed: 1'] = 'price type'

# Use rename_dict to rename nah1 columns
nah1.rename(columns = rename_dict, inplace = True)

# Remove columns where Y is nan
nah1.dropna(subset=['Y'], inplace=True)

# Correct year data
I = nah1['year'].notna()
J = nah1['year'].isna()

nah1.loc[J,['year']] = nah1.loc[I,['year']].values

# Keep rows with '2010-prices, chained values'
I = nah1['price type'] == '2010-prices, chained values'

nah1 = nah1.loc[I,:]

# Only keep renamed variables
nah1 = nah1.loc[:,['year','Y','C','G','I','X','M']]

# Plot the interactive plot
plot_timeseries(nah1)
